In [1]:
ls


In [5]:
cd /home/wsuser/work

/home/wsuser/work


In [3]:
ls

In [4]:
pwd

'/home/wsuser/work'

In [6]:
!pip install tensorflow
!pip install keras

In [8]:
from keras.models import sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='eAxLU8OXmNoxL6DGQDrdWGNTblY9NXxzQxq7rlTN_fIJ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'naturaldisasterintensityanalysisa-donotdelete-pr-1farxsxjugcipc'
object_key = 'dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [10]:
!unzip dataset

unzip:  cannot find or open dataset, dataset.zip or dataset.ZIP.


In [9]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

NameError: name 'streaming_body_1' is not defined

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen=ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,shear_range=0.2)

In [8]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [14]:
x_train=train_datagen.flow_from_directory(r"/dataset/train_set",target_size=(64,64),
                                                            batch_size=5,color_mode='rgb',class_mode='categorical')

Found 742 images belonging to 4 classes.


In [15]:
x_test=test_datagen.flow_from_directory(r"/content/drive/MyDrive/Disaster/dataset/test_set",target_size=(64,64),
                                                            batch_size=5,color_mode='rgb',class_mode="categorical")

Found 198 images belonging to 4 classes.


In [16]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [17]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation="relu",input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=4,activation='softmax'))
model.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer='adam')

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [19]:
model.fit_generator(generator=x_train,epochs=20,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/20
149/149 [==============================] - 42s 279ms/step - loss: 1.2598 - accuracy: 0.4057 - val_loss: 1.1694 - val_accuracy: 0.5152
Epoch 2/20
149/149 [==============================] - 40s 269ms/step - loss: 1.0102 - accuracy: 0.5660 - val_loss: 0.9444 - val_accuracy: 0.5657
Epoch 3/20
149/149 [==============================] - 38s 258ms/step - loss: 0.8004 - accuracy: 0.6658 - val_loss: 1.0515 - val_accuracy: 0.5354
Epoch 4/20
149/149 [==============================] - 38s 259ms/step - loss: 0.6436 - accuracy: 0.7291 - val_loss: 1.2422 - val_accuracy: 0.5000
Epoch 5/20
149/149 [==============================] - 40s 272ms/step - loss: 0.6698 - accuracy: 0.7345 - val_loss: 0.6722 - val_accuracy: 0.7273
Epoch 6/20
149/149 [==============================] - 39s 262ms/step - loss: 0.5201 - accuracy: 0.8086 - val_loss: 0.8071 - val_accuracy: 0.6667
Epoch 7/20
149/149 [==============================] - 39s 260ms/step - loss: 0.5239 - accuracy: 0.7857 - val_loss: 0.7226 - val_ac

In [20]:
model.save('disaster.h5')
model_json=model.to_json()
with open("model-bw.json","w") as json_file:
  json_file.write(model_json)

In [21]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model=load_model('disaster.h5')

In [22]:
x_train.class_indices

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}

In [23]:
img=image.load_img(r"/content/drive/MyDrive/Disaster/dataset/test_set/Earthquake/1329.jpg",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
index=['Cyclone','Earthquake','Flood','Wildfire']
y=np.argmax(model.predict(x),axis=1)
print(index[int(y)])

1/1 [==============================] - 0s 121ms/step
Earthquake


In [24]:
img=image.load_img(r"/content/drive/MyDrive/Disaster/dataset/test_set/Cyclone/900.jpg",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
index=['Cyclone','Earthquake','Flood','Wildfire']
y=np.argmax(model.predict(x),axis=1)
print(index[int(y)])

1/1 [==============================] - 0s 20ms/step
Cyclone
